In [43]:
import numpy as np
from PIL import Image
import ctypes
import matplotlib.pyplot as plt
import random

In [44]:
asia = Image.open('asia.jpg')
greek = Image.open('greek.jpg')
south_america = Image.open('south_america.jpg')

In [45]:
newsize = (27,27)

In [46]:
asia_new_size = asia.resize(newsize)
greek_new_size = greek.resize(newsize)
south_america_new_size = south_america.resize(newsize)

In [47]:
asia_new_size.show()
greek_new_size.show()
south_america_new_size.show()

In [49]:
asia_data = np.asarray(asia_new_size)
greek_data = np.asarray(greek_new_size)
south_america_data = np.asarray(south_america_new_size)

X = np.array([asia_data, greek_data, south_america_data], dtype=ctypes.c_float)
X_flat = X.flatten()

In [8]:
print(len(X_flat))

6561


In [57]:
print(len(X))

3


In [58]:
Y = np.array([[1, -1, -1], [-1, 1, -1], [-1, -1, 1]], dtype=ctypes.c_float)
Y_flat = Y.flatten()
print(Y_flat)

[ 1. -1. -1. -1.  1. -1. -1. -1.  1.]


In [59]:
my_lib = ctypes.CDLL(r"..\rust_lib\target\debug\rust_lib.dll")

my_lib.delete_float_array.argtypes = [
    ctypes.POINTER(ctypes.c_float),
    ctypes.c_int32,
]

my_lib.generate_random_mpl_w.argtypes = [ctypes.POINTER(ctypes.c_float), ctypes.c_int32]
my_lib.generate_random_mpl_w.restype = ctypes.POINTER(ctypes.c_float)

my_lib.delete_float_array.restype = None

layers = np.array([len(X_flat)/len(X), 16, 16, 16, 16, 3], dtype=ctypes.c_float)
LP_c_float = ctypes.POINTER(ctypes.c_float)
layers_pmc = (ctypes.c_float * len(layers))(*layers)
layers_ptr_pmc = ctypes.cast(layers_pmc, LP_c_float)

print(len(layers_pmc))
number_of_layers = len(layers_pmc)
print(layers)

6
[2187.   16.   16.   16.   16.    3.]


In [60]:
w_array_ptr = my_lib.generate_random_mpl_w(layers_ptr_pmc, number_of_layers)
w_array = np.ctypeslib.as_array(w_array_ptr, ((2188*16 + 17*16 + 17*16 +17*16+ 17*3,))) #3*16 + 17*16 + 17*3

LP_c_float = ctypes.POINTER(ctypes.c_float)
x_to_predict = (ctypes.c_float * len(X_flat))(*X_flat)
x_to_predict_ptr = ctypes.cast(x_to_predict, LP_c_float)
LP_c_float = ctypes.POINTER(ctypes.c_float)
y_to_predict = (ctypes.c_float * len(Y_flat))(*Y_flat)
y_to_predict_ptr = ctypes.cast(y_to_predict, LP_c_float)
my_lib.multi_layer_perceptron_training.argtypes = [
    ctypes.POINTER(ctypes.c_float),
    ctypes.POINTER(ctypes.c_float),
    ctypes.POINTER(ctypes.c_float),
    ctypes.c_int32,
    ctypes.c_int32,
    ctypes.c_int32,
    ctypes.c_float,
    ctypes.c_int32,
    ctypes.POINTER(ctypes.c_float),
    ctypes.c_int32,
    ctypes.c_bool,
]
my_lib.multi_layer_perceptron_training.restype = ctypes.POINTER(ctypes.c_float)

In [61]:
pmc_trained_model_ptr = my_lib.multi_layer_perceptron_training(
    w_array_ptr,
    y_to_predict_ptr,
    x_to_predict_ptr,
    len(X),  # number of inputs
    int(len(X_flat)/len(X)),  # dimension of inputs
    3,  # number of classes to predict
    0.1,  # learning rate
    100,  # epochs
    layers_ptr_pmc,  # pointeur vers la forme de pmc
    number_of_layers,  # number of layers
    True,
)

pmc_trained_model = np.ctypeslib.as_array(pmc_trained_model_ptr, (2188*16 + 17*16 + 17*16 +17*16+ 17*3,))
#print(pmc_trained_model)


In [68]:
asia_data = np.asarray(asia_new_size)
greek_data = np.asarray(greek_new_size)
south_america_data = np.asarray(south_america_new_size)

new_X = np.array([south_america_data, asia_data, greek_data], dtype=ctypes.c_float)
new_flat_X = X.flatten()

In [69]:
my_lib.multi_layer_perceptron_predict.argtypes = [
    ctypes.POINTER(ctypes.c_float),
    ctypes.POINTER(ctypes.c_float),
    ctypes.c_int32,
    ctypes.c_int32,
    ctypes.c_int32,
    ctypes.POINTER(ctypes.c_float),
    ctypes.c_int32,
    ctypes.c_bool,
]

my_lib.multi_layer_perceptron_predict.restype = ctypes.POINTER(ctypes.c_float)

LP_c_float = ctypes.POINTER(ctypes.c_float)
x_to_predict_mlp = (ctypes.c_float * len(new_flat_X))(*new_flat_X)
x_to_predict_ptr_mlp = ctypes.cast(x_to_predict_mlp, LP_c_float)

pmc_predict_ptr = my_lib.multi_layer_perceptron_predict(
    pmc_trained_model_ptr,  # pointeur vers le modele entrainé
    x_to_predict_ptr_mlp,  # pointeur vers le dataset à prédire
    len(X),  # nombre d'éléments du dataset
    int(len(X_flat)/len(X)),  # dimension des éléments du dataset
    3,  # number of classes to predict
    layers_ptr_pmc,  # pointeur vers la forme de pmc
    number_of_layers,  # number of layers
    True,  # is classification
)

pmc_predict_model = np.ctypeslib.as_array(pmc_predict_ptr, (3 * len(X),))
# print(pmc_predict_model)
# print(len(X_to_predict))
# print(len(X_to_predict_flat))


In [67]:
print(pmc_predict_model)

[ 0.95705676 -0.9888257  -0.98240566 -0.96874535  0.9717122  -0.9874089
 -0.99191004 -0.95757884  0.9669857 ]


In [70]:
print(pmc_predict_model)

[ 0.95705676 -0.9888257  -0.98240566 -0.96874535  0.9717122  -0.9874089
 -0.99191004 -0.95757884  0.9669857 ]
